# Simulated Annealing for a 3D Skyrmion Lattice

## TODOS

* Recheck computations (signs in delta energy, acceptProb, ...)
* Read and think about parallelization
* Put analysis in place: autocorrelation, phase diagram, etc.
* Best way to setup parameterfile that i can control from the makefile?

In [ ]:
using PyCall
using PyPlot
using HDF5
using StatsBase
# using ParallelAccelerator
# using Distributions
using Base.Profile
using Base.Test
using ProgressMeter

## Parameters & Initialization

In [ ]:
srand(123);

### Type Definitions and Initialization

In [ ]:
type Parameters
    N::Int64 # number of lattice points in each direction
    Ntot::Int64 # total number of lattice points
    Ntherm::Int64 # number of sweeps for thermalization
    Nsweeps::Int64 # number of sweeps between configurations
    Nconfig::Int64 # number of configurations to save
    T::Float64 # the current temperature
    Tmu::Float64 # the reduction factor of the temperature
    Tmin::Float64 # the minimal temperature at which we stop
    J::Float64 # ferromagnetic exchange
    Jp::Float64 # ferromagnetic exchange of correction term
    K::Float64 # DM interaction
    Kp::Float64 # DM interaction of correction term
    B::Float64 # magnetic field in 3 direction
    function Parameters(; N::Int64=10, Ntherm::Int64=3000, Nsweeps::Int64=30, Nconfig::Int64=100, T::Float64=1.0, Tmu::Float64=1.05, Tmin::Float64=1.0e-8, J::Float64=1.0, K::Float64=tan(2pi/10.0), B::Float64=0.0)
        Ntot = N^3
        Jp = J/16.0
        Kp = K/8.0
        new(N, Ntot, Ntherm, Nsweeps, Nconfig, T, Tmu, Tmin, J, Jp, K, Kp, B)
    end
end

In [ ]:
type Monitor
    reject::Int64 # number of rejected steps
    accept::Int64 # number of accepted steps
    Monitor(; reject::Int64=0, accept::Int64=0) = new(reject, accept)
end

In [ ]:
type Results
    E::Vector{Float64} # total energy
    Etmp::Float64 # current total energy
    M::Array{Float64,2} # total magnetization
    Mtmp::Vector{Float64} # current total magnetization
    T::Vector{Float64} # temperature
    os::Int64 # current offset (monte carlo time)
    Results(pars::Parameters) = new(zeros(Float64, pars.Nconfig), 0.0, zeros(Float64, 3, pars.Nconfig), zeros(Float64, 3), zeros(Float64, pars.Nconfig), 0)
end

In [ ]:
function init(pars::Parameters)
    (Monitor(), Results(pars))
end

function init()
    pars = Parameters()
    (pars, init(pars)...)
end

### Grid Initialization

In [ ]:
function uniformS2(dims::Tuple{Int64,Int64,Int64})
    phi = 2pi .* rand(dims)
    u = 2rand(dims) - 1.0
    squ = sqrt(1.0 - u.*u)
    x = squ .* cos(phi)
    y = squ .* sin(phi)
    res = zeros(3,dims...)
    for i=1:*(dims...)
        @inbounds res[:,i] = [x[i] y[i] u[i]]
    end
    res
end

@inline function uniformS2!(s::Vector{Float64})
    phi = 2pi*rand()
    u = 2rand() - 1.0
    squ = sqrt(1.0 - u*u)
    s[1] = squ*cos(phi)
    s[2] = squ*sin(phi)
    s[3] = u
    nothing
end

In [ ]:
function initRandomGrid(dims::Tuple{Int64,Int64,Int64})
    grid = uniformS2(dims)
    res.Etmp = totalEnergy(grid)
    res.Mtmp = totalMagnetization(grid)
    info("Initialized random grid.")
    grid
end

function initRandomGrid()
    initRandomGrid((pars.N, pars.N, pars.N))
end

In [ ]:
@inline idx(i::Int64, N::Int64) = mod(i-1,N)+1

## Hamiltonian

In [ ]:
@inline crossX(a::Vector{Float64}, b::Vector{Float64}) = -a[3]*b[2] + a[2]*b[3]
@inline crossY(a::Vector{Float64}, b::Vector{Float64}) =  a[3]*b[1] - a[1]*b[3]
@inline crossZ(a::Vector{Float64}, b::Vector{Float64}) = -a[2]*b[1] + a[1]*b[2]

In [ ]:
@inline function DMterm(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64, os::Int64)
    const N = pars.N::Int64
    crossX(grid[:,i,j,k], grid[:,idx(i+os,N),j,k]) +
    crossY(grid[:,i,j,k], grid[:,i,idx(j+os,N),k]) +
    crossZ(grid[:,i,j,k], grid[:,i,j,idx(k+os,N)])
end

In [ ]:
function totalEnergy(grid::Array{Float64,4})
    N = pars.N::Int64
    E = 0.0
    for k=1:N, j=1:N, i=1:N
        E += - pars.B * grid[3,i,j,k] -
        pars.J  * dot(grid[:,i,j,k], grid[:,idx(i+1,N),j,k] + grid[:,i,idx(j+1,N),k] + grid[:,i,j,idx(k+1,N)]) +
             pars.Jp * dot(grid[:,i,j,k], grid[:,idx(i+2,N),j,k] + grid[:,i,idx(j+2,N),k] + grid[:,i,j,idx(k+2,N)]) -
             pars.K  * DMterm(grid, i, j, k, 1) +
             pars.Kp * DMterm(grid, i, j, k, 2)
    end
    E
end

In [ ]:
function totalMagnetization(grid::Array{Float64,4})
    N = pars.N::Int64
    M = zeros(Float64, 3)
    for k=1:N, j=1:N, i=1:N
        @inbounds M += grid[:,i,j,k]
    end
    M
end

In [ ]:
function energyChange(grid::Array{Float64,4}, snew::Vector{Float64}, i::Int64, j::Int64, k::Int64)
    const N = pars.N::Int64
    im1 = idx(i-1,N); im2 = idx(i-2,N); ip1 = idx(i+1,N); ip2 = idx(i+2,N)
    jm1 = idx(j-1,N); jm2 = idx(j-2,N); jp1 = idx(j+1,N); jp2 = idx(j+2,N)
    km1 = idx(k-1,N); km2 = idx(k-2,N); kp1 = idx(k+1,N); kp2 = idx(k+2,N)
    sdiff = snew - grid[:,i,j,k]
    -pars.B::Float64 * sdiff[3] -
    pars.J::Float64  * dot(sdiff, grid[:,im1,j,k] + grid[:,ip1,j,k] + grid[:,i,jm1,k] + grid[:,i,jp1,k] + grid[:,i,j,km1] + grid[:,i,j,kp1]) +
    pars.Jp::Float64 * dot(sdiff, grid[:,im2,j,k] + grid[:,ip2,j,k] + grid[:,i,jm2,k] + grid[:,i,jp2,k] + grid[:,i,j,km2] + grid[:,i,j,kp2]) -
    pars.K::Float64  * (crossX(sdiff, grid[:,ip1,j,k] - grid[:,im1,j,k]) +
                        crossY(sdiff, grid[:,i,jp1,k] - grid[:,i,jm1,k]) +
                        crossZ(sdiff, grid[:,i,j,kp1] - grid[:,i,j,km1])) +
    pars.Kp::Float64 * (crossX(sdiff, grid[:,ip2,j,k] - grid[:,im2,j,k]) +
                        crossY(sdiff, grid[:,i,jp2,k] - grid[:,i,jm2,k]) +
                        crossZ(sdiff, grid[:,i,j,kp2] - grid[:,i,j,km2]))
end

## Simulated Annealing

In [ ]:
@inline stepT!() = (pars.T::Float64 /= pars.Tmu::Float64)

In [ ]:
@inline acceptProb(del::Float64, T::Float64) = exp(-del/T)

In [ ]:
function acceptMove!(grid::Array{Float64,4}, snew::Vector{Float64}, dE::Float64, i::Int64, j::Int64, k::Int64)
    mon.accept::Int64 += 1
    res.Etmp::Float64 += dE
    res.Mtmp::Vector{Float64} += snew - grid[:,i,j,k]
    grid[:,i,j,k] = snew
end

In [ ]:
function update!(grid::Array{Float64,4}, snew::Vector{Float64})
    i,j,k = rand(1:pars.N::Int64, 3)
    uniformS2!(snew)
    dE = energyChange(grid, snew, i, j, k)
    if dE < 0.0 || acceptProb(dE, pars.T::Float64) >= rand()
        acceptMove!(grid, snew, dE, i, j, k)
    else
        mon.reject::Int64 += 1
    end
end

In [ ]:
@inline function sweep!(grid::Array{Float64,4}, snew::Vector{Float64})
    for i=1:pars.Ntot::Int64
        update!(grid, snew)
    end
end

In [ ]:
function nextconfig!(grid::Array{Float64,4})
    snew = zeros(Float64, 3)
    for j=1:pars.Nsweeps::Int64
        sweep!(grid, snew)
    end
end

In [ ]:
function thermalize!(grid::Array{Float64,4})
    snew = zeros(Float64, 3)
    @showprogress 1 "Thermalizing..." for i=1:pars.Ntherm::Int64
        sweep!(grid, snew)
    end
    info("The system has thermalized.")
end

function thermalize()
    grid = initRandomGrid()
    thermalize!(grid)
    grid
end

In [ ]:
function run!(grid::Array{Float64,4}, filename::ASCIIString)
    thermalize!(grid)
    h5open(getPath(filename), "w") do file
        conf = d_create(file, "configs", datatype(Float64), dataspace(3,pars.N,pars.N,pars.N,pars.Nconfig),"chunk",(3,pars.N,pars.N,pars.N,1))
        avg = d_create(file, "avg", datatype(Float64), dataspace(3,pars.N,pars.N,pars.N))
        snapshot(grid, conf, avg)
        @showprogress 1 "Main run..." for i = 1:pars.Nconfig::Int64-1
            nextconfig!(grid)
            snapshot(grid, conf, avg)
            # stepT!()
            if 0.0 < pars.T < pars.Tmin
                info("Cooled below minimal temperature: $(pars.Tmin)")
                break
            end
        end
        finalOutput(file, avg)
    end
    info("Finished run.")
    grid
end

function run(filename::ASCIIString)
    grid = initRandomGrid()
    run!(grid, filename)
end

## Output

In [ ]:
function snapshot(grid::Array{Float64,4}, conf::HDF5Dataset, avg::HDF5Dataset)
    res.os::Int64 += 1
    res.E[res.os] = res.Etmp::Float64
    res.M[:,res.os] = res.Mtmp::Vector{Float64}
    res.T[res.os] = pars.T::Float64
    appendConfig(grid, res.os, conf)
    addConfig(grid, avg)
end

In [ ]:
function finalOutput(file::HDF5File, avg::HDF5Dataset)
    file["energy"] = res.E
    file["magnetization"] = res.M
    file["temperature"] = res.T
    writeParameters(file)
    averageConfigs(avg)
end

In [ ]:
function writeParameters(file::HDF5File)
    file["parameters/N"] = pars.N
    file["parameters/Ntherm"] = pars.Ntherm
    file["parameters/Nsweeps"] = pars.Nsweeps
    file["parameters/Nconfig"] = pars.Nconfig
    file["parameters/J"] = pars.J
    file["parameters/Jp"] = pars.Jp
    file["parameters/K"] = pars.K
    file["parameters/Kp"] = pars.Kp
end

In [ ]:
@inline function averageConfigs(avg::HDF5Dataset)
    avg[:,:,:,:] /= pars.Nconfig
end

In [ ]:
@inline function appendConfig(grid::Array{Float64,4}, os::Int64, conf::HDF5Dataset)
    conf[:,:,:,:,os] = grid[:,:,:,:]
end

In [ ]:
@inline function addConfig(grid::Array{Float64,4}, avg::HDF5Dataset)
    avg[:,:,:,:] += grid[:,:,:,:]
end

In [ ]:
@inline function getPath(filename::ASCIIString)
    string("../../data/", filename, ".h5")
end

In [ ]:
@inline function writeArray(data::Array{Float64}, filename::ASCIIString, dataset::ASCIIString=filename)
    h5write(getPath(filename), dataset, data)
end

In [ ]:
@inline function writeGrid(grid::Array{Float64,4}, filename::ASCIIString)
    writeArray(grid, filename, "grid")
end

## Postprocessing

### Basic Statistics

In [ ]:
function variance_meansq(f::Array{Float64}; mean=nothing, meansq=nothing)
    if meansq == nothing
        if mean == nothing
            mean::Float64 = Base.mean(f)
        end
        meansq = mean*mean
    end
    (sumabs2(f)/length(f) - meansq, meansq)
end

In [ ]:
function autocorr(f::Vector{Float64})
    meansq, var = variance_meansq(f)
    N = length(f)
    auto = zeros(Float64, N)
    for t=0:N-1
        tmp = 0.0
        for i=1:N-t
            tmp += f[i]*f[i+t]
        end
        auto[t+1] = (tmp/(N-t) - meansq) / var
    end
    auto
end

In [ ]:
function intAutoTime(f::Vector{Float64}, N::Int64; isautocorr=false)
    if !isautocorr
        f = autocorr(f)
    end
    0.5 + sum(f[2:N])
end

function intAutoTime(f::Vector{Float64}; isautocorr=false)
    intAutoTime(f, length(f), isautocorr=isautocorr)
end

In [ ]:
# experimental, still flawed
function findIntAutoTime(f::Vector{Float64})
    auto = autocorr(f)
    I = 2:fld(length(f),5)
    ts = zeros(size(I))
    for (i, tau) in enumerate(I)
        ts[i] = intAutoTime(auto, 5tau, isautocorr=true)
    end
    val, i = findmin(abs(ts-tau))
    (i, ts)
end

### Skyrmion Specifics

In [ ]:
@inline getX(grid::Array{Float64}) = squeeze(grid[1,:],1)
@inline getY(grid::Array{Float64}) = squeeze(grid[2,:],1)
@inline getZ(grid::Array{Float64}) = squeeze(grid[3,:],1)

In [ ]:
@inline averageSpins(M::Array{Float64,2}) = M/pars.Ntot
@inline averageSpins!(M::Array{Float64,2}) = (M /= pars.Ntot; nothing)

In [ ]:
function braggIntensity(S::Array{Float64,2})
end

In [ ]:
function fft3(f::Array{Float64,3}, N::Int64)
    FFTW.set_num_threads(4)
    fft(f,(N, N, N))
end

## Run the Simulation

In [ ]:
pars = Parameters(N=30, Ntherm=5000, Nsweeps=30, Nconfig=2000, T=0.0, B=0.0)
mon, res = init(pars);
pars.T, pars.B, pars.J, pars.K, pars.N, pars.Ntherm, pars.Nconfig

In [ ]:
@time grid = run("T$(pars.T)_N$(pars.N)");

In [ ]:
tMC = 1:pars.Nconfig
plot(tMC, res.E)
figure()
plot(tMC, getX(res.M), tMC, getY(res.M), tMC, getZ(res.M))

In [ ]:
plot(autocor(res.E,0:size(res.E,1)-1))
figure()
plot(tMC, autocor(getX(res.M),0:pars.Nconfig-1),tMC, autocor(getY(res.M),0:pars.Nconfig-1),tMC, autocor(getZ(res.M),0:pars.Nconfig-1))

In [ ]:
# Profile.clear()
# Profile.init()
# @time run();

In [ ]:
# data, lidict = Profile.retrieve();

## Tests

In [ ]:
function testuniformS2()
    N = 50
    dims = (N, N, N)
    grid = uniformS2(dims);
    for i=1:*(dims...)
        @test norm(grid[:,i]) ≈ 1.0
    end
    println("finished test: uniform points on S2")
end

In [ ]:
function test()
    testuniformS2()
    println("If you do not see any errors, all tests passed.")
end